In [1]:
## import modules
from splinter import Browser
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [2]:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
# Build query Url and List of Stocks ticker
base_url = "https://finance.yahoo.com/quote/"
tickers = ["AAPL", "FB", "IBM", "MSFT",'TSLA', 'AAPL', 'ROKU', 'BA', 'ABEV', 'LYFT', 'VALE']

In [4]:
# create and initialize list 
close_price_list = []
volume_list = []
mkt_cap_list = []
earnings_Date_list = []
eps_list = []

In [5]:
for ticker in tickers:
    try:
        # Visit Yahoo finance summary page
        url1 = base_url + ticker + '?p=' + ticker + '&.tsrc=fin-srch'
    #     print(url1)
        browser.visit(url1)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")
        summary = soup.find('table', class_="W(100%)").find_all(class_="Trsdu(0.3s) ")
        market_info = soup.find('table', class_="W(100%) M(0) Bdcl(c)").find_all(class_="Trsdu(0.3s) ")
        #print(summary)
        #type(summary)
        close_price = soup.find('div', class_="D(ib) Mend(20px)").text
    #     print(close_price.split('+')[0])
        close_price = summary[0].contents[0]
        close_price_list.append(close_price)
        volume = summary[4].contents[0]
        volume_list.append(volume)
        market_cap = market_info[0].contents[0]
        mkt_cap_list.append(market_cap)
        earnings_Date = market_info[4].contents[0]
        earnings_Date_list.append(earnings_Date)
        eps = market_info[3].contents[0]
        eps_list.append(eps)
    
    except AttributeError:
        pass
    except IndexError:
        pass

In [6]:
# remove B and convert Mkt_cap to Float
new_cap_list = []
for cap in mkt_cap_list:
    try:
        new_cap = cap.replace('B', '')
        new_cap = float(new_cap)
        new_cap_list.append(new_cap)
    except ValueError:
        pass
mkt_cap_list = new_cap_list
print(mkt_cap_list)

[961.257, 509.098, 124.875, 946.521, 47.198, 961.257, 6.421, 214.543, 68.563, 19.697, 67.973]


In [7]:
# Fix empty date values with a data
new_d_list = []
# print(earnings_Date_list)
# print("/n ---------------- /n")
for d in earnings_Date_list:
    new_d = d.replace('N/A', "2019-02-07")
    new_d_list.append(new_d)
earnings_Date_list = new_d_list
# print(new_d_list)

In [8]:
# remove comma from Volume values
new_vol_list = []
for vol in volume_list:
    try:
        new_vol = vol.replace(',', '')
        new_vol = float(new_vol)
        new_vol_list.append(new_vol)
    except AttributeError:
        pass
volume_list = new_vol_list
# print(volume_list)

In [9]:
# type(earnings_Date_list)

In [10]:
# create and initialize list 
company_name_list = []
company_desc_list = []
company_tel_list = []
company_addr_list = []
key_execs_n_list = []
key_execs_t_list = []
key_execs_s_list = []
url_list = []

In [11]:
for ticker in tickers:
    try:
        # Visit Yahoo finance profile page
        url2 = base_url + ticker + '/profile?p=' + ticker
    #     print(url2)
        browser.visit(url2)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")

        profile1 = soup.find('p', class_="D(ib) W(47.727%) Pend(40px)")
        company_addr = profile1.contents[1] + profile1.contents[5]
        company_addr_list.append(company_addr)
    #     print(company_addr)
        #type(data)
        company_name = soup.find("h3", class_="Fz(m) Mb(10px)").text
        company_name_list.append(company_name)
        company_desc = soup.find('p', class_="Mt(15px) Lh(1.6)").text
        company_desc_list.append(company_desc)
    #     print(company_desc)
        key_execs_data = soup.find('table', class_='W(100%)').find('tbody').find_all('span')
#         print(key_execs_data[0])
        key_execs_name = key_execs_data[0].contents[1]
        key_execs_title = key_execs_data[1].contents[1]
        key_execs_salary = key_execs_data[2].contents[1] 
        if len(key_execs_salary):
            key_execs_salary = 'N/A'
        key_execs_n_list.append(key_execs_name)
        key_execs_t_list.append(key_execs_title)
        key_execs_s_list.append(key_execs_salary)
        profile = soup.find('p', class_="D(ib) W(47.727%) Pend(40px)").find_all('a')
        company_tel = profile[0].text
        url = profile[1].text
        company_tel_list.append(company_tel)
        url_list.append(url)
        
    except IndexError as e:
        pass
    except AttributeError as e:
        pass

In [12]:
#print(key_execs_s_list)

In [13]:
# convert telephone number to Float
new_tel_list = []
for tel in company_tel_list:
    try:
        new_tel = tel.replace('-', '')
        new_tel = float(new_tel)
        new_tel_list.append(new_tel)
    except ValueError:
        pass
company_tel_list = new_tel_list
#print(company_tel_list)

In [14]:
# remove M and convert Key_Execs_Salary to Float
new_s_list = []
for s in key_execs_s_list:
    try:
        new_s = s.replace('M', '')
        new_s = float(new_s)
        new_s_list.append(new_s)
        
    except ValueError:
        pass
    
key_execs_s_list = new_s_list
# print(key_execs_s_list)

In [15]:
# create and initialize list 
total_ESG_score_list = []
avg_perf_env_list = []
social_score_list = []
governance_score_list = []

In [16]:
for ticker in tickers:
    try:
        # Visit Yahoo finance sustanability page
        url3 = base_url + ticker + '/sustainability?p=' + ticker
    #     print(url3)
        browser.visit(url3)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")
        sustain = soup.find_all('div', class_="D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)")
    #     print(sustain[count].contents[0])
    #     count += 1
        total_ESG_score = soup.find('div', class_="Fz(36px) Fw(600) D(ib) Mend(5px)").text
        total_ESG_score_list.append(total_ESG_score)
        avg_perf_env = sustain[0].contents[0]
        avg_perf_env_list.append(avg_perf_env)
        social_score = sustain[1].contents[0]
        social_score_list.append(social_score)
        governance_score = sustain[2].contents[0]
        governance_score_list.append(governance_score)
        
    except AttributeError as e:
        pass

In [17]:
# Create Summary dataframe
data2_dic = {'Ticker':tickers, 'Close_Price':close_price_list, 'Volume':volume_list,'Mkt_Cap':mkt_cap_list, 'EPS':eps_list, 'Earnings_Date':earnings_Date_list}
summary_df = pd.DataFrame.from_dict(data2_dic, orient='index')
summary_df = summary_df.transpose()

In [18]:
summary_df

,Ticker,Close_Price,Volume,Mkt_Cap,EPS,Earnings_Date
0,AAPL,203.13,2.41958e+07,961.257,12.12,2019-02-08
1,FB,178.78,1.16556e+07,509.098,7.57,2019-02-07
2,IBM,139.11,4.70475e+06,124.875,9.50,2019-02-07
3,MSFT,121.77,2.7991e+07,946.521,4.31,2019-05-15
4,TSLA,271.23,5.87632e+06,47.198,-5.72,2019-02-07
5,AAPL,203.13,2.41958e+07,961.257,12.12,2019-02-08
6,ROKU,56.60,9.48068e+06,6.421,-0.09,2019-02-07
7,BA,377.52,4.32314e+06,214.543,17.85,2019-02-07
8,ABEV,4.3400,2.46369e+07,68.563,0.1760,2018-12-19
9,LYFT,59.51,6.72887e+06,19.697,-43.04,2019-02-07


In [50]:
# Create Summary dataframe
data_dic = {'Ticker':tickers, 'Company_Name':company_name_list, 'Description':company_desc_list, 'Telephone':company_tel_list, 'Address':company_addr_list,'Key_Execs_Name':key_execs_n_list,'Key_Execs_Title':key_execs_t_list,'Web_URL':url_list}
profile_df = pd.DataFrame.from_dict(data_dic, orient='index')
profile_df = profile_df.transpose()

In [57]:
# Combine Dataframe
profile_df = profile_df.drop(columns=['Ticker'])
summary_profile_combine_df = pd.concat([profile_df, summary_df], axis=1, sort=True)
summary_profile_combine_df

,Company_Name,Description,Telephone,Address,Key_Execs_Name,Key_Execs_Title,Web_URL,Ticker,Close_Price,Volume,Mkt_Cap,EPS,Earnings_Date
0,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",4.08996e+09,"One Apple Park WayCupertino, CA 95014",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com,AAPL,203.13,2.41958e+07,961.257,12.12,2019-02-08
1,"Facebook, Inc.","Facebook, Inc. provides various products to co...",6.50543e+09,"1601 Willow RoadMenlo Park, CA 94025",Mr. Mark Elliot Zuckerberg,"Founder, Chairman & CEO",http://www.facebook.com,FB,178.78,1.16556e+07,509.098,7.57,2019-02-07
2,International Business Machines Corporation,International Business Machines Corporation op...,9.14499e+09,"1 New Orchard RoadArmonk, NY 10504",Ms. Virginia M. Rometty,"Chairman, Pres & CEO",http://www.ibm.com,IBM,139.11,4.70475e+06,124.875,9.50,2019-02-07
3,Microsoft Corporation,"Microsoft Corporation develops, licenses, and ...",4.25883e+09,"One Microsoft WayRedmond, WA 98052",Mr. Satya Nadella,CEO & Director,http://www.microsoft.com,MSFT,121.77,2.7991e+07,946.521,4.31,2019-05-15
4,"Tesla, Inc.","Tesla, Inc. designs, develops, manufactures, a...",4.08996e+09,"3500 Deer Creek RoadPalo Alto, CA 94304",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com,TSLA,271.23,5.87632e+06,47.198,-5.72,2019-02-07
5,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",4.08557e+09,"One Apple Park WayCupertino, CA 95014",Mr. Anthony J. Wood,"Founder, Chairman, Pres & CEO",http://www.roku.com,AAPL,203.13,2.41958e+07,961.257,12.12,2019-02-08
6,"Roku, Inc.","Roku, Inc. operates a TV streaming platform. T...",3.12544e+09,"150 Winchester CircleLos Gatos, CA 95032",Mr. Dennis A. Muilenburg,"Chairman, Pres & CEO",http://www.boeing.com,ROKU,56.60,9.48068e+06,6.421,-0.09,2019-02-07
7,The Boeing Company,"The Boeing Company, together with its subsidia...",None,"100 North Riverside PlazaChicago, IL 60606",None,None,None,BA,377.52,4.32314e+06,214.543,17.85,2019-02-07
8,Ambev S.A.,"Ambev S.A., through its subsidiaries, produces...",None,"Rua Dr. Renato Paes de Barros, 10173rd Floor",None,None,None,ABEV,4.3400,2.46369e+07,68.563,0.1760,2018-12-19
9,"Lyft, Inc.","Lyft, Inc. operates a peer-to-peer marketplace...",None,185 Berry StreetSuite 5000,None,None,None,LYFT,59.51,6.72887e+06,19.697,-43.04,2019-02-07


In [35]:
# Create sustainability dataframe
data1_dic = {'Ticker':tickers, 'Total_ESG_Score':total_ESG_score_list, 'AVG_Perf_Env':avg_perf_env_list,'Social_Score':social_score_list, 'Governance_Score':governance_score_list}
sustainability_df = pd.DataFrame.from_dict(data1_dic, orient='index')
sustainability_df = sustainability_df.transpose()

In [59]:
# # Add sustainability to dataframe
sustainability_df = sustainability_df.drop(columns=['Ticker'])
summary_profile_sustainability_df = pd.concat([summary_profile_combine_df, sustainability_df], axis=1, sort=True)
summary_profile_sustainability_df

,Company_Name,Description,Telephone,Address,Key_Execs_Name,Key_Execs_Title,Web_URL,Ticker,Close_Price,Volume,Mkt_Cap,EPS,Earnings_Date,Total_ESG_Score,AVG_Perf_Env,Social_Score,Governance_Score
0,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",4.08996e+09,"One Apple Park WayCupertino, CA 95014",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com,AAPL,203.13,2.41958e+07,961.257,12.12,2019-02-08,70,89,57,60
1,"Facebook, Inc.","Facebook, Inc. provides various products to co...",6.50543e+09,"1601 Willow RoadMenlo Park, CA 94025",Mr. Mark Elliot Zuckerberg,"Founder, Chairman & CEO",http://www.facebook.com,FB,178.78,1.16556e+07,509.098,7.57,2019-02-07,59,62,67,42
2,International Business Machines Corporation,International Business Machines Corporation op...,9.14499e+09,"1 New Orchard RoadArmonk, NY 10504",Ms. Virginia M. Rometty,"Chairman, Pres & CEO",http://www.ibm.com,IBM,139.11,4.70475e+06,124.875,9.50,2019-02-07,74,87,74,60
3,Microsoft Corporation,"Microsoft Corporation develops, licenses, and ...",4.25883e+09,"One Microsoft WayRedmond, WA 98052",Mr. Satya Nadella,CEO & Director,http://www.microsoft.com,MSFT,121.77,2.7991e+07,946.521,4.31,2019-05-15,74,82,71,70
4,"Tesla, Inc.","Tesla, Inc. designs, develops, manufactures, a...",4.08996e+09,"3500 Deer Creek RoadPalo Alto, CA 94304",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com,TSLA,271.23,5.87632e+06,47.198,-5.72,2019-02-07,60,70,49,55
5,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",4.08557e+09,"One Apple Park WayCupertino, CA 95014",Mr. Anthony J. Wood,"Founder, Chairman, Pres & CEO",http://www.roku.com,AAPL,203.13,2.41958e+07,961.257,12.12,2019-02-08,70,89,57,60
6,"Roku, Inc.","Roku, Inc. operates a TV streaming platform. T...",3.12544e+09,"150 Winchester CircleLos Gatos, CA 95032",Mr. Dennis A. Muilenburg,"Chairman, Pres & CEO",http://www.boeing.com,ROKU,56.60,9.48068e+06,6.421,-0.09,2019-02-07,51,54,43,58
7,The Boeing Company,"The Boeing Company, together with its subsidia...",None,"100 North Riverside PlazaChicago, IL 60606",None,None,None,BA,377.52,4.32314e+06,214.543,17.85,2019-02-07,None,None,None,None
8,Ambev S.A.,"Ambev S.A., through its subsidiaries, produces...",None,"Rua Dr. Renato Paes de Barros, 10173rd Floor",None,None,None,ABEV,4.3400,2.46369e+07,68.563,0.1760,2018-12-19,None,None,None,None
9,"Lyft, Inc.","Lyft, Inc. operates a peer-to-peer marketplace...",None,185 Berry StreetSuite 5000,None,None,None,LYFT,59.51,6.72887e+06,19.697,-43.04,2019-02-07,None,None,None,None


In [60]:
# Database 
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
rds_connection_string = "root:root123@127.0.0.1:3306/twitter_stocks_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [61]:
engine.table_names()

['profile', 'stock_data', 'summary', 'sustainability']

In [62]:
# converted DataFrame into database
summary_profile_sustainability_df.to_sql(name='stock_data', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from stock_data', con=engine).head()

,Company_Name,Description,Telephone,Address,Key_Execs_Name,Key_Execs_Title,Web_URL,Ticker,Close_Price,Volume,Mkt_Cap,EPS,Earnings_Date,Total_ESG_Score,AVG_Perf_Env,Social_Score,Governance_Score
0,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",4.089961e+09,"One Apple Park WayCupertino, CA 95014",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com,AAPL,203.13,24195766.0,961.257,12.12,2019-02-08,70,89,57,60
1,"Facebook, Inc.","Facebook, Inc. provides various products to co...",6.505435e+09,"1601 Willow RoadMenlo Park, CA 94025",Mr. Mark Elliot Zuckerberg,"Founder, Chairman & CEO",http://www.facebook.com,FB,178.78,11655608.0,509.098,7.57,2019-02-07,59,62,67,42
2,International Business Machines Corporation,International Business Machines Corporation op...,9.144992e+09,"1 New Orchard RoadArmonk, NY 10504",Ms. Virginia M. Rometty,"Chairman, Pres & CEO",http://www.ibm.com,IBM,139.11,4704746.0,124.875,9.50,2019-02-07,74,87,74,60
3,Microsoft Corporation,"Microsoft Corporation develops, licenses, and ...",4.258828e+09,"One Microsoft WayRedmond, WA 98052",Mr. Satya Nadella,CEO & Director,http://www.microsoft.com,MSFT,121.77,27990998.0,946.521,4.31,2019-05-15,74,82,71,70
4,"Tesla, Inc.","Tesla, Inc. designs, develops, manufactures, a...",4.089961e+09,"3500 Deer Creek RoadPalo Alto, CA 94304",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com,TSLA,271.23,5876325.0,47.198,-5.72,2019-02-07,60,70,49,55


In [64]:
# converted DataFrame into database
summary_df.to_sql(name='summary', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from summary', con=engine).head()

,Ticker,Close_Price,Volume,Mkt_Cap,EPS,Earnings_Date
0,AAPL,203.13,24195766.0,961.257,12.12,2019-02-08
1,FB,178.78,11655608.0,509.098,7.57,2019-02-07
2,IBM,139.11,4704746.0,124.875,9.50,2019-02-07
3,MSFT,121.77,27990998.0,946.521,4.31,2019-05-15
4,TSLA,271.23,5876325.0,47.198,-5.72,2019-02-07


In [65]:
# converted DataFrame into database
# pd.to_sql('TRUNCATE TABLE profile', con=engine)
profile_df.to_sql(name='profile', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from profile', con=engine).head()

,Company_Name,Description,Telephone,Address,Key_Execs_Name,Key_Execs_Title,Web_URL
0,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",4.089961e+09,"One Apple Park WayCupertino, CA 95014",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com
1,"Facebook, Inc.","Facebook, Inc. provides various products to co...",6.505435e+09,"1601 Willow RoadMenlo Park, CA 94025",Mr. Mark Elliot Zuckerberg,"Founder, Chairman & CEO",http://www.facebook.com
2,International Business Machines Corporation,International Business Machines Corporation op...,9.144992e+09,"1 New Orchard RoadArmonk, NY 10504",Ms. Virginia M. Rometty,"Chairman, Pres & CEO",http://www.ibm.com
3,Microsoft Corporation,"Microsoft Corporation develops, licenses, and ...",4.258828e+09,"One Microsoft WayRedmond, WA 98052",Mr. Satya Nadella,CEO & Director,http://www.microsoft.com
4,"Tesla, Inc.","Tesla, Inc. designs, develops, manufactures, a...",4.089961e+09,"3500 Deer Creek RoadPalo Alto, CA 94304",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com


In [66]:
# converted DataFrame into database
sustainability_df.to_sql(name='sustainability', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from sustainability', con=engine).head()

,Total_ESG_Score,AVG_Perf_Env,Social_Score,Governance_Score
0,70,89,57,60
1,59,62,67,42
2,74,87,74,60
3,74,82,71,70
4,60,70,49,55
